In [2]:
import random
import time
import threading

In [3]:
def sleep(inter_num, interval):
    for i in range(inter_num):
        print("I am alive {}".format(i))
        time.sleep(interval)
t = threading.Thread(target=sleep, args=(5, 1))
t.daemon = True
t.start()

I am alive 0
I am alive 1
I am alive 2
I am alive 3
I am alive 4


In [29]:
# TODO: давайте починим output

def sleep(inter_num , interval, lock):
    for i in range(inter_num):
        with lock:
            print("I am alive {}".format(i))
        time.sleep(0)

inter_num = 5
interval = 1
lock = threading.Lock()
t = threading.Thread(target=sleep, args=(inter_num, interval, lock))
t.start()

for i in range(inter_num):
    with lock:
        print("I am boss {}".format(i))
    time.sleep(interval)

I am alive 0
I am alive 1
I am alive 2
I am boss 0
I am alive 3
I am alive 4
I am boss 1
I am boss 2
I am boss 3
I am boss 4


In [6]:
class Sleeper(threading.Thread):
    def __init__(self, name, inter_num, interval):
        super().__init__()
        self.name = name
        self.iter_num = inter_num
        self.interval = interval
    
    def run(self):
        for i in range(self.iter_num):
            print("I am alive {} my name {}".format(i, self.name))
            time.sleep(self.interval)

inter_num = 5
interval = 1
            
t = Sleeper("Ilya", inter_num, interval)
t.start()

for i in range(inter_num):
    print("I am boss {}".format(i))
    time.sleep(interval)

I am alive 0 my name IlyaI am boss 0

I am alive 1 my name IlyaI am boss 1

I am alive 2 my name IlyaI am boss 2

I am alive 3 my name IlyaI am boss 3

I am alive 4 my name IlyaI am boss 4



In [35]:
# TODO: Создадим 6 потоков по числу преподов Vlad, Zarina, Misha, Ilya, Vlad, Anton
preps = ["Vlad","Zarina", "Misha", 'Ilya', "vlad","anton"]
lock = threading.Lock()
def greet(name):
    with lock:
        print("My name is {}".format(name))
for prep in preps:
    t = threading.Thread(target=greet, args=(prep,))
    t.start()
    

My name is Vlad
My name is Zarina
My name is Misha
My name is Ilya
My name is vlad
My name is anton


In [34]:
# join
class Sleeper(threading.Thread):
    def __init__(self, name, inter_num, interval):
        super().__init__()
        self.name = name
        self.iter_num = inter_num
        self.interval = interval
    
    def run(self):
        for i in range(self.iter_num):
            print("I am alife {} my name {}".format(i, self.name))
            time.sleep(self.interval)

inter_num = 5
interval = 1
            
t = Sleeper("Ilya", inter_num, interval)
t.start()
t.join()

for i in range(inter_num):
    print("I am boss {}".format(i))
    time.sleep(interval)

I am alife 0 my name Ilya
I am alife 1 my name Ilya
I am alife 2 my name Ilya
I am alife 3 my name Ilya
I am alife 4 my name Ilya
I am boss 0
I am boss 1
I am boss 2
I am boss 3
I am boss 4


In [ ]:
# TODO: Заставим преподов из задачки выше говорить последовательно

In [8]:
class DollarsAccount:
    def __init__(self, dollars_amount):
        self._dollars = dollars_amount

In [11]:
class Inflation(threading.Thread):
    def __init__(self, name, account, amount, iter_num):
        super().__init__()
        self.name = name
        self.account = account
        self.amount = amount
        self.iter_num = iter_num
    
    def run(self):
        print("My name {} account id {}".format(self.name, id(self.amount)))
        for i in range(self.iter_num):
            self.account._dollars -= self.amount
            
class DepositCap(threading.Thread):
    def __init__(self, name, account, amount, iter_num):
        super().__init__()
        self.name = name
        self.account = account
        self.amount = amount
        self.iter_num = iter_num
    
    def run(self):
        print("My name {} account id {}".format(self.name, id(self.amount)))
        for i in range(self.iter_num):
            self.account._dollars += self.amount

In [15]:
dac = DollarsAccount(50)
inflation = Inflation("inflation", dac, 1, 100000)      
deposit = DepositCap("DepositCap", dac, 1, 100000)
 
inflation.start()
deposit.start()
inflation.join()
deposit.join()
print(dac._dollars)

My name inflation account id 4394190240
My name DepositCap account id 4394190240
-7915


In [16]:
class Inflation(threading.Thread):
    def __init__(self, name, account, amount, iter_num):
        super().__init__()
        self.name = name
        self.account = account
        self.amount = amount
        self.iter_num = iter_num
        self.lock = threading.Lock()
    
    def run(self):
        print("My name {} account id {}".format(self.name, id(self.amount)))
        for i in range(self.iter_num):
            self.lock.acquire()
            self.account._dollars -= self.amount
            self.lock.release()
            
class DepositCap(threading.Thread):
    def __init__(self, name, account, amount, iter_num):
        super().__init__()
        self.name = name
        self.account = account
        self.amount = amount
        self.iter_num = iter_num
        self.lock = threading.Lock()
    
    def run(self):
        print("My name {} account id {}".format(self.name, id(self.amount)))
        for i in range(self.iter_num):
            self.lock.acquire()
            self.account._dollars += self.amount
            self.lock.release()

In [18]:
dac = DollarsAccount(50)
inflation = Inflation("inflation", dac, 5, 100000)      
deposit = DepositCap("DepositCap", dac, 5, 100000)

inflation.start()
deposit.start()
inflation.join()
deposit.join()
print(dac._dollars)

#Почему не сработало?

My name inflation account id 4394190368
My name DepositCap account id 4394190368
135285


In [20]:
class Inflation(threading.Thread):
    def __init__(self, name, account, amount, iter_num, lock):
        super().__init__()
        self.name = name
        self.account = account
        self.amount = amount
        self.iter_num = iter_num
        self.lock = lock
    
    def run(self):
        print("My name {} account id {}".format(self.name, id(self.amount)))
        for i in range(self.iter_num):
            with self.lock:
                self.account._dollars -= self.amount
            
class DepositCap(threading.Thread):
    def __init__(self, name, account, amount, iter_num, lock):
        super().__init__()
        self.name = name
        self.account = account
        self.amount = amount
        self.iter_num = iter_num
        self.lock = lock
    
    def run(self):
        print("My name {} account id {}".format(self.name, id(self.amount)))
        for i in range(self.iter_num):
            with self.lock:
                self.account._dollars += self.amount
            
dac = DollarsAccount(50)
lock = threading.Lock()
inflation = Inflation("inflation", dac, 5, 100000, lock)      
deposit = DepositCap("DepositCap", dac, 5, 100000, lock)

inflation.start()
deposit.start()
inflation.join()
deposit.join()
print(dac._dollars)

My name inflation account id 4394190368
My name DepositCap account id 4394190368
50


In [39]:
import threading, time

a = 5
alock = threading.Lock()
b = 5
block = threading.Lock()

def thread1calc():
    global a
    global b
    print("Thread1 acquiring lock a")
    alock.acquire()
    time.sleep(5)

    print("Thread1 acquiring lock b")
    block.acquire()
    time.sleep(5)
    a += 5
    b += 5

    print("Thread1 releasing both locks")
    block.release()
    alock.release()

def thread2calc():
    global a
    global b
    print ("Thread2 acquiring lock b")
    alock.acquire()
    time.sleep(5)

    print("Thread2 acquiring lock a")
    block.acquire()
    time.sleep(5)
    a += 10
    b += 10

    print("Thread2 releasing both locks")
    block.release()
    alock.release()

t1 = threading.Thread(target = thread1calc)
t1.start()

t2 = threading.Thread(target = thread2calc)
t2.start()

t1.join()
t2.join()

Thread1 acquiring lock a
Thread2 acquiring lock b
Thread1 acquiring lock b
Thread1 releasing both locks
Thread2 acquiring lock a
Thread2 releasing both locks
